# Практическое задание 1

# Ранжирование вопросов StackOverflow с помощью векторных представлений слов

## курс "Математические методы анализа текстов"


### ФИО: Власов Андрей Валерьевич

## Введение

В этом задании вы научитесь вычислять близость текстов и применить этот метод для поиска похожих вопросов на [StackOverflow](https://stackoverflow.com).

### Используемые библиотеки

В данном задании потребуются следующие библиотеки:
- [Gensim](https://radimrehurek.com/gensim/) — инструмент для решения различных задач NLP (тематическое моделирование, представление текстов, ...).
- [Numpy](http://www.numpy.org) — библиотека для научных вычислений.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — библилиотека с многими реализованными алгоритмами машинного обучения для анализа данных.
- [Nltk](http://www.nltk.org) — инструмент для работы с естественными языками.

Для выполнения бонусной части потребуется:
- [StarSpace](https://github.com/facebookresearch/StarSpace) — универсальная модель для обучения различных векторных представлений, разработанная командой Facebook.


### Данные

Данные лежат в архиве `StackOverflowData.zip`, который состоит из:
- `train.tsv` - обучающая выборка. В каждой строке через табуляцию записаны дублирующие друг друга предложения;
- `test.tsv` - тестовая выборка. В каждой строке через табуляцию записаны: *<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...*

Скачать архив можно здесь: [ссылка на google диск](https://drive.google.com/open?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_)

### Вектора слов/

Для решения вам потребуются две модели векторных представлений слов:

 - [Предобученные векторные представления слов](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit), которые были обучены с помощью стандартной модели word2vec на данных Google News (100 миллиардов слов). Модель содержит 300-мерные вектора для 3 миллионов слов и фраз. Вы можете скачать их, запустив блок кода ниже.
 - Векторные представления слов, полученные с помощью StarSpace на выборке StackOverflow. Вам потребуется обучить эту модель самим во второй части задания.

In [ ]:
# Download Google vectors to directory *target_dir*

from download_utils import download_google_vectors
download_google_vectors(target_dir='.')

## Часть 1. Предобученные векторные представления слов (2 балла)

Скачайте предобученные вектора и загрузите их с помощью функции [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) библиотеки Gensim с параметром *binary=True*. Если суммарный размер векторов больше, чем доступная память, то вы можете загрузите только часть векторов, указав параметр *limit* (рекомендуемое значение: 500000).

In [1]:
import gensim

In [2]:
wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', 
                                                             binary=True, limit=500000)
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

### Как пользоваться этими векторами?

Как только вы загрузите векторные представления слов в память, убедитесь, что имеете к ним доступ. Сначала вы можете проверить, содержится ли какое-то слово в загруженных эмбедингах:

    'word' in wv_embeddings

Затем, чтобы получить соответствующий вектор, вы можете использовать оператор доступа по ключу:

    wv_embeddings['word']

### Проверим, корректны ли векторные представления

Чтобы предотвратить возможные ошибки во время первого этапа, можно проверить, что загруженные вектора корректны. Для этого вы можете запустить функцию *check_embeddings*. Она запускает 3 теста:
1. Находит наиболее похожие слова для заданных "положительных" и "отрицательных" слов.
2. Находит, какое слово из заданного списка не встречается с остальными.
3. Находит наиболее похожее слово для заданного.

In [4]:
from tests import check_embeddings
print(check_embeddings(wv_embeddings))

These embeddings look good.


/home/andrey/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:858: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


### Векторные представления текста

Чтобы перейти от отдельных слов к векторным представлениям вопросов, предлагается подсчитать **среднее** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектоора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор. 

<font color='red'>**Реализуйте функцию *question_to_vec_by_mean*, работающую по такой логике. **</font>

In [5]:
import numpy as np

In [6]:
def question_to_vec_by_mean(question, embeddings, dim=300):
    """
        question: a string
        embeddings: dict where the key is a word and a value is its' embedding
        dim: size of the representation

        result: vector representation for the question
    """
    words = question.split(' ')
    vectors = []
    for word in words:
            if word in embeddings:
                vectors.append(embeddings[word])
    if len(vectors) == 0:
        vectors.append(np.zeros(dim))
    result = np.mean(vectors, axis=0)
    
    return result
    ##########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ 
    ##########################

Для базовой проверки решения запустите клетку ниже.

In [7]:
from tests import question_to_vec_tests
print(question_to_vec_tests(question_to_vec_by_mean, wv_embeddings))

Basic tests are passed.


In [8]:
question_to_vec_by_mean('word', wv_embeddings)

array([ 3.59375000e-01,  4.15039062e-02,  9.03320312e-02,  5.46875000e-02,
       -1.47460938e-01,  4.76074219e-02, -8.49609375e-02, -2.04101562e-01,
        3.10546875e-01, -1.05590820e-02, -6.15234375e-02, -1.55273438e-01,
       -1.52343750e-01,  8.54492188e-02, -2.70996094e-02,  3.84765625e-01,
        4.78515625e-02,  2.58789062e-02,  4.49218750e-02, -2.79296875e-01,
        9.09423828e-03,  4.08203125e-01,  2.40234375e-01, -3.06640625e-01,
       -1.80664062e-01,  4.73632812e-02, -2.63671875e-01,  9.08203125e-02,
        1.37695312e-01, -7.20977783e-04,  2.67333984e-02,  1.92382812e-01,
       -2.29492188e-02,  9.70458984e-03, -7.37304688e-02,  4.29687500e-01,
       -7.93457031e-03,  1.06445312e-01,  2.80761719e-02, -2.29492188e-01,
       -1.91650391e-02, -2.36816406e-02,  3.51562500e-02,  1.71875000e-01,
       -1.12304688e-01,  6.25000000e-02, -1.69921875e-01,  1.29882812e-01,
       -1.54296875e-01,  1.58203125e-01, -7.76367188e-02,  1.78710938e-01,
       -1.72851562e-01,  

Теперь у нас есть метод для создания векторного представления любого предложения. Оценим, как будет работать это решение.

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из *N* вопросов *R* случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели *R + 1* примеров и смотреть на позицию дубликата.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)],$$
где $q_i$ - $i$-ый вопрос, $dup_i$ - его дубликат, $topK(q_i)$ - первые *K* элементов в ранжированном списке, который выдает наша модель.

#### DCG@K
Второй метрикой будет упрощенная [DCG метрика](https://en.wikipedia.org/wiki/Discounted_cumulative_gain):
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K],$$
где $rank_{dup_i}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$. С такой метрикой модель штрафуется за низкую позицию корректного ответа.

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. Пусть $N = 1$, $R = 3$, вопрос $q_1$ это "Что такое python", а его дубликат $dup_1$ это "Что такое язык python". Пусть модель выдала следующий ранжированный список кандидатов:

1. *"Как узнать с++"*
2. *"Что такое язык python"*
3. *"Хочу учить Java"*
4. *"Не понимаю Tensorflow"*

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [dup_1 \in top1(q_1)] = 0$
- [K = 4] $\text{Hits@4} =  [dup_1 \in top4(q_1)] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

<font color='red'>**Реализуйте функции *hits_count* и *dcg_score*. **</font> 

Каждая функция имеет два аргумента: *dup_ranks* и *k*. *dup_ranks* является списком, который содржит *рейтинги дубликатов* (их позиции в ранжированном списке). Например, *dup_ranks = [2]* для примера, описанного выше.

In [9]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list of ranks of the duplicates; one rank per question; 
                   length is a number of questions that we check (N); 
                   rank is a number from 1 to len(candidates for the question).
        k: number of top-ranked elements (k in Hits@k metric)

        result: return Hits@k value for the current ranking.
    """
    cnt = 0
    for rank in dup_ranks:
        if rank <= k:
            cnt+=1
    result = cnt/ len(dup_ranks) 
    return result
    ############################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ############################

In [10]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list of ranks of the duplicates; one rank per question; 
                   length is a number of questions that we check (N); 
                   rank is a number from 1 to len(candidates for the question).
        k: number of top-ranked elements (k in DCG@k metric)

        result: return DCG@k value for the current ranking.
    """
    cnt= 0
    for rank in dup_ranks:
        if rank <= k:
            cnt+=1/np.log2(1+rank)
    result = cnt/ len(dup_ranks) 
    return result 
    ############################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ############################

Протестируйте функции. Успешное прохождение базовых тестов еще не гарантирует корректности реализации!

In [11]:
from tests import test_hits
print(test_hits(hits_count))

Basic test are passed.


In [12]:
from tests import test_dcg
print(test_dcg(dcg_score))

Basic test are passed.


### Ранжирование вопросов StackOverflow

Выборка уже разбита на обучающую и тестовую. Все файлы используют табуляцию в качестве разделителя, но они имеют разный формат:

- *обучающая* выборка (test.tsv) содержит похожие друг на друга предложения в каждой строке;
- *тестовая* выборка (validation.tsv) содержит в каждой строке: *вопрос, похожий вопрос, отрицательный пример 1, отрицательный пример 2, ...*

Считайте тестовую выборку для оценки качества текущего решения.

In [13]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

In [14]:
import pandas as pd

In [15]:
validation = read_corpus('data/validation.tsv')

<font color='red'>**Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния.**</font>
    
Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список *[(2, c), (0, a), (1, b)]*.

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
def rank_candidates(question, candidates, embeddings, dim=300):
    """
        question: a string
        candidates: a list of strings (candidates) which we want to rank
        embeddings: some embeddings
        dim: dimension of the current embeddings
        
        result: a list of pairs (initial position in the list, question)
    """
    similarities = []
    result = []
    question = question_to_vec_by_mean(question,embeddings,dim).reshape(1,-1)
    for rank,card in enumerate(candidates):
        card_old = card
        card = question_to_vec_by_mean(card,embeddings,dim).reshape(1,-1)
        similarities.append((cosine_similarity(question,card),rank,card_old))
    similarities = sorted(similarities,reverse=True)
    for similarity in similarities:
        result.append((similarity[1],similarity[2]) )
    return result
    
    pass

Протестируйте работу функции на примерах ниже.

In [21]:
from tests import test_rank_candidates
print(test_rank_candidates(rank_candidates, wv_embeddings))

Basic tests are passed.


Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут).

In [23]:
wv_ranking = []
for line in validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [24]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.203 | Hits@   1: 0.203
DCG@   5: 0.259 | Hits@   5: 0.308
DCG@  10: 0.277 | Hits@  10: 0.362
DCG@ 100: 0.316 | Hits@ 100: 0.560
DCG@ 500: 0.349 | Hits@ 500: 0.817
DCG@1000: 0.368 | Hits@1000: 1.000


Если вы проделали все шаги правильно, то вы должны разочароваться полученными результатами. Давайте попробуем понять, почему качество модели такое низкое. Когда вы работаете с какими-либо данными, очень полезно первым делом посмотреть на них глазами. Выведите несколько вопросов из наших данных:

In [25]:
for line in validation[:3]:
    q, *examples = line
    print(q, *examples[:3])
    print()

How to print a binary heap tree without recursion? How do you best convert a recursive function to an iterative one? How can i use ng-model with directive in angular js flash: drawing and erasing

How to start PhoneStateListener programmatically? PhoneStateListener and service Java cast object[] to model WCF and What does this mean?

jQuery: Show a div2 when mousenter over div1 is over when hover on div1 depenting on if it is on div2 or not it should act differently How to run selenium in google app engine/cloud? Python Comparing two lists of strings for similarities



## Часть 2. Предобработка данных (1 балл)

Как вы могли заметить, мы имеем дело с сырыми данными. Это означает, что там присутствует много опечаток, спецсимволов и заглавных букв. В нашем случае это все может привести к ситуации, когда для данных токенов нет предобученных векторов. Поэтому необходима предобработка.

<font color='red'>**Реализуйте функцию предобработки текстов.**</font>

Вам требуется:
- Перевести символы в нижний регистр;
- Заменить символы пунктуации на пробелы;
- Удалить "плохие" символы;
- Удалить стопслова.

In [22]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/andrey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
stopwords_eng = stopwords.words('english')
reg = nltk.tokenize.RegexpTokenizer(r'\w+') 
    
def text_prepare(text):
    """
        text: a string
        
        return: modified string
    """
    text = text.lower()
    text = reg.tokenize(text)
    text = " ".join([word for word in text if word not in stopwords_eng])
    return text
    
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################

<font color='red'>**Теперь преобразуйте все вопросы из тестовой выборки. Оцените, как изменилось качество. Сделайте выводы.**</font>

In [25]:
from tqdm import tqdm_notebook

In [26]:
new_validation = []

for questions in tqdm_notebook(validation):
    blocks = []
    for quest in questions:
        blocks.append(text_prepare(quest))  
    new_validation.append(blocks)

100%|██████████| 3760/3760 [00:57<00:00, 65.85it/s]


In [34]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
wv_ranking_new = []
for line in tqdm_notebook(new_validation):
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking_new.append([r[0] for r in ranks].index(0) + 1)

100%|██████████| 3760/3760 [07:22<00:00,  8.49it/s]


In [35]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking_new, k), k, hits_count(wv_ranking_new, k)))

DCG@   1: 0.325 | Hits@   1: 0.325
DCG@   5: 0.402 | Hits@   5: 0.470
DCG@  10: 0.417 | Hits@  10: 0.517
DCG@ 100: 0.453 | Hits@ 100: 0.695
DCG@ 500: 0.473 | Hits@ 500: 0.850
DCG@1000: 0.488 | Hits@1000: 1.000


## Часть 3. Представления для неизвестных слов. (2 балла)

<font color='red'>**Оцените долю слов в выборке, для которых нет эмбеддинга в модели.**</font>

In [116]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
def calc_proportion(validation,embeddings):
    not_emb = []
    emb = []
    all_words = []
    for questions in validation:
        for quest in questions:
            words = quest.split(' ')
            for word in words:
                all_words.append(word)
                if word not in embeddings:
                    not_emb.append(word)
                else:
                    emb.append(word)
    return not_emb,emb,all_words

In [117]:
not_emb,emb,all_words = calc_proportion(validation,wv_embeddings)
print(len(set(not_emb)),len(not_emb),len(set(all_words)),len(all_words))
len(set(not_emb))/len(set(all_words)),len(not_emb)/len(all_words)

26633 8283101 38538 32106478


(0.6910841247599772, 0.2579884657544811)

In [118]:
not_emb,emb,all_words = calc_proportion(new_validation,wv_embeddings)
print(len(set(not_emb)),len(not_emb),len(set(all_words)),len(all_words))
len(set(not_emb))/len(set(all_words)),len(not_emb)/len(all_words)

10233 3385749 18166 22495084


(0.5633050754156116, 0.15051061823107661)

In [121]:
emb = list(set(emb))

Для того, что получить представления для неизвестного слова, воспользуемся следующим подходом:
    
1. Будем восстанавливать эмбеддинг неизвестного слова как сумму эмбеддингов буквенных триграмм. Например, слово where должно представляться суммой триграмм _wh, whe, her, ere, re_

2. В качестве обучающих данных будем использовать слова, для которых есть эмбеддинг в модели. Будем обучать эмбеддинги триграмм по выборке эмбеддингов с помощью функционала MSE:

$$L = \sum_{w \in W_{known}}\| f_{\theta}(w) - v_w \|^2 \to \min_{\theta}$$

где:

* $W_{known}$ — множество известных модели слов
* $f_{\theta}(w)$ — сумма эмбеддингов триграмм слова $w$
* $v_w$ — эмбеддинг слова $w$
* $\theta$ — веса эмбеддингов триграмм

<font color='red'>**Реализуйте предложенную модель ниже.**</font>

Используйте класс nn.EmbeddingBag для построения среднего вектора представлений.

In [34]:
wv_vocab = list(wv_embeddings.vocab)

In [492]:
wv_vocab_all = {}
for key,value in tqdm(zip(wv_vocab,wv_embeddings.vectors)):
    wv_vocab_all[key]=value


500000it [00:00, 2019863.93it/s]


In [276]:
wv_vocab_all['in']

array([ 0.0703125 ,  0.08691406,  0.08789062,  0.0625    ,  0.06933594,
       -0.10888672, -0.08154297, -0.15429688,  0.02075195,  0.13183594,
       -0.11376953, -0.03735352,  0.06933594,  0.078125  , -0.10302734,
       -0.09765625,  0.04418945,  0.10253906, -0.06079102, -0.03613281,
       -0.04541016,  0.04736328, -0.12060547, -0.06396484,  0.0022583 ,
        0.03710938, -0.00291443,  0.11767578,  0.06176758,  0.06396484,
        0.08105469, -0.06884766, -0.0213623 ,  0.05517578, -0.08544922,
        0.06884766, -0.12792969, -0.03320312,  0.09863281,  0.17578125,
        0.11083984, -0.03466797, -0.04711914, -0.00848389,  0.03588867,
        0.10302734,  0.02697754, -0.02868652, -0.00512695,  0.10644531,
        0.05981445,  0.09423828,  0.03369141, -0.02709961, -0.09423828,
        0.00102997, -0.04833984,  0.03442383,  0.08105469, -0.11328125,
       -0.08886719,  0.03588867, -0.14550781, -0.24414062, -0.06152344,
        0.05297852,  0.05688477,  0.1796875 ,  0.06103516,  0.08

In [277]:
check = " where "
trigrams_check = [ (check[i]+ check[i + 1]+ check[i + 2])
           for i in range(len(check) - 2)]
trigrams_check

[' wh', 'whe', 'her', 'ere', 're ']

In [278]:
check = "precompile"
trigrams_check = [ (check[i]+ check[i + 1]+ check[i + 2])
           for i in range(len(check) - 2)]
if len(check)>2:
    trigrams_check.append(check[:2])
    trigrams_check.append(check[-2:])
if len(check) ==2 & (len(check[:2])==len(check[:-2])):
    trigrams_check.append(check[:2])
trigrams_check

['pre', 'rec', 'eco', 'com', 'omp', 'mpi', 'pil', 'ile', 'pr', 'le']

In [293]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [340]:
def make_trigrams(wv_word):
    trigrams = [ (wv_word[i]+ wv_word[i + 1]+ wv_word[i + 2]) for i in range(len(wv_word) - 2)]
    if len(wv_word)>2:
        trigrams.append(wv_word[:2])
        trigrams.append(wv_word[-2:])
    if len(wv_word) ==2 :
        trigrams.append(wv_word[:2])
    if len(wv_word) ==1 :
        trigrams.append(wv_word[:2])
    return trigrams

In [482]:
class TrigrammEmbeddingsModel(nn.Module):
    
    def __init__(self, all_known_tokens, embedding_dim=300):
        """
        all_known_tokens : list of str
        
        embedding_dim : int
        """
        super(TrigrammEmbeddingsModel, self).__init__()
        
        emb_trigrams = []
            
        for w in all_known_tokens:
            lst_trig = make_trigrams(w)
            for trig in lst_trig:
                emb_trigrams.append(trig)

                
        self.emb_trigrams = list(set(emb_trigrams))

        self.embedding = nn.EmbeddingBag(len(self.emb_trigrams),embedding_dim, mode='sum')
        
        self.word_to_ix = {word: i for i, word in enumerate(self.emb_trigrams)}
        
        del emb_trigrams
        

    def forward(self, token):
        trigrams = make_trigrams(token)
        
        context_idxs = torch.tensor([word_to_ix[w] for w in trigrams if w in self.emb_trigrams], dtype=torch.long)        
        embedded = self.embedding(context_idxs,torch.tensor([0], dtype=torch.long))
        return embedded
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################


<font color='red'>** Обучите модель. Оцените, как изменилось качество. Сделайте выводы.**</font>

Если вы всё реализовали правильно, качество решения должно вырасти.

In [483]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
loss_function = nn.MSELoss(reduction='sum')
model = TrigrammEmbeddingsModel(emb)
optimizer = optim.SGD(model.parameters(),lr=1e-4)

In [484]:
len(emb)

7933

In [511]:
from torch.optim.lr_scheduler import StepLR

In [513]:
losses=[]
scheduler10 = StepLR(optimizer, step_size=10, gamma=0.1)
for epoch in tqdm_notebook(range(15)):
    total_loss = 0
    for word_indx, word in tqdm_notebook(enumerate(emb)):
        trigramm_emb_sum = model(word)

        loss = loss_function(trigramm_emb_sum, torch.tensor([wv_embeddings[word]], dtype=torch.float32))
        optimizer.zero_grad()

    #         if word_indx % 100 == 0:
    #             print(f'word {word} number {word_indx} of {len(emb)}, loss {loss.item()}' )

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    scheduler10.step()
    total_loss = total_loss/len(emb)
    losses.append(total_loss)
print(losses) 

[1299.5714678252823, 1268.3225867140268, 1238.8211302147167, 1210.900431827866, 1184.4173349788987, 1159.2476492996823, 1135.282774521426, 1112.427084687768, 1090.5958682817243, 1069.7136967614638, 1049.713094825509, 1037.605618326237, 1035.7000789981344, 1033.8023634775582, 1031.9124205630287]


In [515]:
model('recursion')

tensor([[ 4.0226, -3.2949,  0.5479,  1.0509, -3.9814,  4.4350, -0.1516, -0.5238,
          2.1994, -2.3503, -1.9312,  3.7317, -3.3457, -0.2678, -1.3114, -0.0884,
          2.7414,  0.1291, -0.4851,  1.2593, -0.4343, -0.5687, -0.9252, -2.2965,
         -1.1011, -2.9368,  0.4264, -0.4570,  2.8129, -2.6712,  2.1259,  1.6930,
          0.0507,  1.1553,  0.7067,  3.9731, -1.5501, -1.7003,  0.1955,  1.2425,
         -1.3060,  1.9096,  2.1361,  2.1357, -0.5932,  1.6000,  0.6950, -2.0958,
          0.8221,  1.5572, -1.4201, -1.3339, -0.1526, -0.6479, -0.2864, -1.4138,
          1.1786,  3.2894,  0.9411, -1.6700, -2.7773,  2.0487,  1.8302,  0.5926,
         -1.2815,  0.5591, -5.9965,  1.7289,  2.5235, -0.5404, -0.1636,  0.3937,
         -1.8177, -2.6313, -0.8833,  2.6657, -1.9473,  1.0772,  1.5346,  3.4262,
         -0.3042, -1.3921,  2.4929, -4.6760,  2.3037, -3.0958,  2.4653,  2.6036,
         -0.7576, -0.9465,  0.9372,  1.4302, -0.4282,  1.8936, -0.1807,  1.1568,
         -1.2189,  0.2614,  

In [516]:
onemore_wv = {}
for em in tqdm_notebook(set(not_emb)):
    onemore_wv[em] =  model(em).detach().numpy()

In [518]:
wv_vocab_all = {}
for key,value in tqdm(zip(wv_vocab,wv_embeddings.vectors)):
    wv_vocab_all[key]=value



0it [00:00, ?it/s]
204603it [00:00, 2039147.77it/s]
393742it [00:00, 1965469.13it/s]
500000it [00:00, 2009724.95it/s]

In [519]:
wv_vocab_all.update(onemore_wv)

In [520]:
wv_ranking_onemore = []
for line in tqdm_notebook(new_validation):
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_vocab_all)
    wv_ranking_onemore.append([r[0] for r in ranks].index(0) + 1)

In [521]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking_onemore, k), k, hits_count(wv_ranking_onemore, k)))

DCG@   1: 0.378 | Hits@   1: 0.378
DCG@   5: 0.447 | Hits@   5: 0.502
DCG@  10: 0.459 | Hits@  10: 0.540
DCG@ 100: 0.482 | Hits@ 100: 0.655
DCG@ 500: 0.505 | Hits@ 500: 0.841
DCG@1000: 0.522 | Hits@1000: 1.000


In [509]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking_onemore, k), k, hits_count(wv_ranking_onemore, k)))

DCG@   1: 0.377 | Hits@   1: 0.377
DCG@   5: 0.444 | Hits@   5: 0.499
DCG@  10: 0.455 | Hits@  10: 0.532
DCG@ 100: 0.479 | Hits@ 100: 0.651
DCG@ 500: 0.502 | Hits@ 500: 0.840
DCG@1000: 0.519 | Hits@1000: 1.000


In [496]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking_onemore, k), k, hits_count(wv_ranking_onemore, k)))

DCG@   1: 0.374 | Hits@   1: 0.374
DCG@   5: 0.442 | Hits@   5: 0.499
DCG@  10: 0.452 | Hits@  10: 0.529
DCG@ 100: 0.475 | Hits@ 100: 0.646
DCG@ 500: 0.499 | Hits@ 500: 0.835
DCG@1000: 0.516 | Hits@1000: 1.000


## Бонусная часть: векторные представления StarSpace (2 балла)

В бонусной части вам предлгается обучить эмбеддинги специально для задачи поиска дубликатов с помощью пакета [StarSpace](https://github.com/facebookresearch/StarSpace). К сожалению, его нельзя запустить на Windows, поэтому в этом случае мы рекоммендуем использовать готовый [docker container](https://github.com/hse-aml/natural-language-processing/blob/master/Docker-tutorial.md) с пошаговыми инструкциями или воспользоваться платформой google colab.

Данная модель все еще представляет вопросы с помощью усреднения векторов слов, однако обучается по размеченной выборке пар близких вопросов. Это позволяет обучить вектора, которые лучше подходят для конкретной задачи. Напомним, что в модели word2vec обучение происходят по парам близких слов, и на этапе обучения модель ничего не знает о наших планах по их усреднению в пост-обработке.


### Как выбрать  параметры модели?

Ниже приведены некоторые рекомендации, с которых можно начать свои эксперименты.

- Обучение на парах близких предложений соответствует режиму *trainMode = 3*.
- Используйте метод оптимизации adagrad (параметр *adagrad=True*).
- Установите длину фразы равной 1 (параметр *ngrams*), чтобы получить только вектора слов.
- Не используйте большое количество эпох (5 должно быть достаточно).
- Поэкспериментируйте с несколькими размерностями *dim* (например, от 100 до 300).
- Для сравнения векторов используйте *косинусную меру*.
- Установите *minCount* больше 1 (например, 2), если вы не хотите получить вектора для редко встречающихся слов.
- Параметр *verbose=True* будет показывать вам прогресс процесса обучения.
- Параметр *negSearchLimit* отвечает за число отрицательных примеров, которые используются в обучении, рекомендованное значение 10.
- Для ускорения обучения мы рекоммендуем поставить *шаг обучения (learning rate)* равным 0.05.

<font color='red'>** Обучите вектора StarSpace для униграм на обучающей выборке. Не забудьте использовать предобработанную версию данных. **</font>

Если вы следовали инструкциям правильно, то процесс обучения займет около 1 часа. Размер словаря полученных векторных представлений должен быть порядка 100000 (число строк в полученном файле). 

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data
validation = read_corpus('data/validation.tsv')

In [132]:
def new_file(old_file, new_file):
    new = open(new_file, 'w')
    for questions in open(old_file, encoding='utf8'):
        questions = questions.strip().split('\t')
        new_questions = []
        for ques in questions:
            new_questions.append(text_prepare(ques))
        print(*new_questions, sep='\t', file=new)
    new.close()

In [134]:
new_file('data/train.tsv', 'data/new_train.tsv')

In [142]:
##### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ TRAINING HAPPENING HERE #####
!./starspace train -trainFile "data/train.tsv" -model StarSpace_emb -trainMode 3 -adagrad true -ngrams 1 \
-epoch 5 -dim 100 -similarity cosine -minCount 2 -verbose true  -negSearchLimit 10 -lr 0.05 -fileFormat labelDoc

Arguments: 
lr: 0.05
dim: 100
epoch: 5
maxTrainTime: 8640000
validationPatience: 10
saveEveryEpoch: 0
loss: hinge
margin: 0.05
similarity: cosine
maxNegSamples: 10
negSearchLimit: 10
batchSize: 5
thread: 10
minCount: 2
minCountLabel: 1
label: __label__
label: __label__
ngrams: 1
bucket: 2000000
adagrad: 1
trainMode: 3
fileFormat: labelDoc
normalizeText: 0
dropoutLHS: 0
dropoutRHS: 0
useWeight: 0
weightSep: :
Start to initialize starspace model.
Build dict from input file : data/train.tsv
Read 19M words
Number of words in dictionary:  205221
Number of labels in dictionary: 0
Loading data from file : data/train.tsv
Total number of examples loaded : 999799
Initialized model weights. Model size :
matrix : 205221 100
Training epoch 0: 0.05 0.01
Epoch: 100.0%  lr: 0.040040  loss: 0.061931  eta: 0h3m  tot: 0h0m47s  (20.0%).074012  eta: 0h3m  tot: 0h0m32s  (13.0%)0.073338  eta: 0h3m  tot: 0h0m32s  (13.3%)
 ---+++                Epoch    0 Train error : 0.06212483 +++--- ☃
Training epoch 1: 0.0

In [176]:
##### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ TRAINING HAPPENING HERE #####
!./starspace train -trainFile "data/new_train.tsv" -model StarSpace_emb_new -trainMode 3 -adagrad true -ngrams 1 \
-epoch 5 -dim 100 -similarity cosine -minCount 2 -verbose true  -negSearchLimit 10 -lr 0.05 -fileFormat labelDoc

Arguments: 
lr: 0.05
dim: 100
epoch: 5
maxTrainTime: 8640000
validationPatience: 10
saveEveryEpoch: 0
loss: hinge
margin: 0.05
similarity: cosine
maxNegSamples: 10
negSearchLimit: 10
batchSize: 5
thread: 10
minCount: 2
minCountLabel: 1
label: __label__
label: __label__
ngrams: 1
bucket: 2000000
adagrad: 1
trainMode: 3
fileFormat: labelDoc
normalizeText: 0
dropoutLHS: 0
dropoutRHS: 0
useWeight: 0
weightSep: :
Start to initialize starspace model.
Build dict from input file : data/new_train.tsv
Read 13M words
Number of words in dictionary:  73320
Number of labels in dictionary: 0
Loading data from file : data/new_train.tsv
Total number of examples loaded : 999910
Initialized model weights. Model size :
matrix : 73320 100
Training epoch 0: 0.05 0.01
Epoch: 100.0%  lr: 0.040000  loss: 0.038340  eta: 0h2m  tot: 0h0m36s  (20.0%) tot: 0h0m2s  (1.4%)0.070352  eta: 0h3m  tot: 0h0m7s  (3.7%)m  tot: 0h0m18s  (9.7%)2m  tot: 0h0m21s  (11.3%)0.044184  loss: 0.045964  eta: 0h2m  tot: 0h0m22s  (11.8%)


In [522]:
##### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ TRAINING HAPPENING HERE #####
!./starspace train -trainFile "data/new_train.tsv" -model StarSpace_emb_new_300 -trainMode 3 -adagrad true -ngrams 1 \
-epoch 5 -dim 300 -similarity cosine -minCount 2 -verbose true  -negSearchLimit 10 -lr 0.05 -fileFormat labelDoc

Arguments: 
lr: 0.05
dim: 300
epoch: 5
maxTrainTime: 8640000
validationPatience: 10
saveEveryEpoch: 0
loss: hinge
margin: 0.05
similarity: cosine
maxNegSamples: 10
negSearchLimit: 10
batchSize: 5
thread: 10
minCount: 2
minCountLabel: 1
label: __label__
label: __label__
ngrams: 1
bucket: 2000000
adagrad: 1
trainMode: 3
fileFormat: labelDoc
normalizeText: 0
dropoutLHS: 0
dropoutRHS: 0
useWeight: 0
weightSep: :
Start to initialize starspace model.
Build dict from input file : data/new_train.tsv
Read 13M words
Number of words in dictionary:  73320
Number of labels in dictionary: 0
Loading data from file : data/new_train.tsv
Total number of examples loaded : 999910
Initialized model weights. Model size :
matrix : 73320 300
Training epoch 0: 0.05 0.01
Epoch: 100.0%  lr: 0.040040  loss: 0.038182  eta: 0h8m  tot: 0h2m12s  (20.0%)  lr: 0.047608  loss: 0.066575  eta: 0h12m  tot: 0h0m37s  (4.7%)0.047247  loss: 0.062591  eta: 0h11m  tot: 0h0m41s  (5.5%)%  lr: 0.046036  loss: 0.054014  eta: 0h10m  

Ниже вы можете проверить качество работы вашей модели. Так как обучение происходило для конкретной задачи на размеченных данных, то ожидается, что это решение будет иметь более высокое качество.

In [525]:
def read_starspace(filename):
    starspace_embedding = {}
    for line in open(filename, encoding='utf-8'):
        line = line.strip().split('\t')
        starspace_embedding[line[0]] = np.array(line[1:]).astype(np.float32)
    return starspace_embedding
starspace_embeddings = read_starspace('StarSpace_emb.tsv')

In [526]:
starspace_embeddings_new = read_starspace('StarSpace_emb_new.tsv')

In [527]:
starspace_embeddings_new_300 = read_starspace('StarSpace_emb_new_300.tsv')

In [169]:
ss_prepared_ranking = []
for line in tqdm(new_validation):
    q, *ex = line
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ss_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [170]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(ss_prepared_ranking, k), 
                                              k, hits_count(ss_prepared_ranking, k)))

DCG@   1: 0.473 | Hits@   1: 0.473
DCG@   5: 0.578 | Hits@   5: 0.668
DCG@  10: 0.597 | Hits@  10: 0.728
DCG@ 100: 0.630 | Hits@ 100: 0.887
DCG@ 500: 0.642 | Hits@ 500: 0.977
DCG@1000: 0.644 | Hits@1000: 1.000


In [528]:
ss_prepared_ranking_new = []
for line in tqdm_notebook(new_validation):
    q, *ex = line
    ranks = rank_candidates(q, ex, starspace_embeddings_new, 100)
    ss_prepared_ranking_new.append([r[0] for r in ranks].index(0) + 1)

In [532]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(ss_prepared_ranking_new, k), 
                                              k, hits_count(ss_prepared_ranking_new, k)))

DCG@   1: 0.528 | Hits@   1: 0.528
DCG@   5: 0.633 | Hits@   5: 0.720
DCG@  10: 0.652 | Hits@  10: 0.778
DCG@ 100: 0.682 | Hits@ 100: 0.925
DCG@ 500: 0.690 | Hits@ 500: 0.985
DCG@1000: 0.692 | Hits@1000: 1.000


In [530]:
ss_prepared_ranking_new_300 = []
for line in tqdm_notebook(new_validation):
    q, *ex = line
    ranks = rank_candidates(q, ex, starspace_embeddings_new_300, 300)
    ss_prepared_ranking_new_300.append([r[0] for r in ranks].index(0) + 1)

In [531]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(ss_prepared_ranking_new_300, k), 
                                              k, hits_count(ss_prepared_ranking_new_300, k)))

DCG@   1: 0.527 | Hits@   1: 0.527
DCG@   5: 0.638 | Hits@   5: 0.729
DCG@  10: 0.656 | Hits@  10: 0.785
DCG@ 100: 0.685 | Hits@ 100: 0.923
DCG@ 500: 0.693 | Hits@ 500: 0.985
DCG@1000: 0.694 | Hits@1000: 1.000
